Lambda School Data Science

*Unit 2, Sprint 2, Module 1*

---

In [1]:
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    !pip install category_encoders==2.*
    !pip install pandas-profiling==2.*

DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'

     |████████████████████████████████| 81kB 3.8MB/s 
     |████████████████████████████████| 266kB 5.5MB/s 
     |████████████████████████████████| 604kB 25.9MB/s 
     |████████████████████████████████| 71kB 9.3MB/s 
     |████████████████████████████████| 3.1MB 31.2MB/s 
     |████████████████████████████████| 71kB 9.3MB/s 
     |████████████████████████████████| 71kB 9.3MB/s 
     |████████████████████████████████| 296kB 46.1MB/s 
  Created wheel for htmlmin: filename=htmlmin-0.1.12-cp36-none-any.whl size=27085 sha256=369eb95335ecfb8be8de35b1583c0fe27a7ed0ae6d7558aa715269627f0be0b2
  Stored in directory: /root/.cache/pip/wheels/43/07/ac/7c5a9d708d65247ac1f94066cf1db075540b85716c30255459
  Created wheel for imagehash: filename=ImageHash-4.1.0-py2.py3-none-any.whl size=291991 sha256=61f2f27d45e171bf66fa940399f2c0415175bb3ca7f95e81dfeb6183e91b74bf
  Stored in directory: /root/.cache/pip/wheels/07/1c/dc/6831446f09feb8cc199ec73a0f2f0703253f6ae013a22f4be9
Successfully built htmlmin image

# Decision Trees

## Kaggle

**Task 1:** [Sign up for a Kaggle account](https://www.kaggle.com/), if you don’t already have one. Go to our Kaggle InClass competition website (the URL is in Slack). Go to the Rules page. Accept the rules of the competition. Notice that the Rules page also has instructions for the Submission process. The Data page has feature definitions.

## Wrangle Data

**Task 2:** Add to the code below so that `id` is set as the index for `df`.

In [137]:
import pandas as pd

df = pd.merge(pd.read_csv(DATA_PATH + 'waterpumps/train_features.csv'),
              pd.read_csv(DATA_PATH + 'waterpumps/train_labels.csv'))

df.index = df.id  
df = df.drop(columns=['id'])

df.head()

,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional
67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional
19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional


**Test 2**

In [7]:
'''Task 2 Testing'''

assert isinstance(df, pd.DataFrame), 'Have you created the DataFrame `df`?'
assert df.shape == (59400, 40), '`df` is the wrong shape. Did you set the index to `id`?'
assert 69572 in df.index, 'The index for `df` is has the wrong values. Did you set the index to `id`?'

## Split Data

**Task 3:** Create your target vector `y` and feature matrix `X`.

In [138]:
target = 'status_group'

y = df[target]
X = df.drop(columns=[target])
print('Shapes of X and y', X.shape, y.shape)

Shapes of X and y (59400, 39) (59400,)


**Test 3**

In [134]:
'''Task 3 Testing'''

assert isinstance(X, pd.DataFrame), '`X` is the wrong data type.'
assert isinstance(y, pd.Series), '`y` is the wrong data type.'
assert y.shape == (59400,), '`y` is the wrong shape.'
assert X.shape == (59400,39), '`X` is the wrong shape.'

In [139]:
def little_wrangle(X):

  # Prevent SettingWithCopyWarning
  X = X.copy()

  # Construction Year
  def date_or_nan(year):
    if year == 0:
      return np.nan
    else:
      return datetime.datetime(int(year), 1, 1)
  X['construction_year'] = X['construction_year'].apply(date_or_nan)

  # Inspection Date
  X['date_recorded'] = pd.to_datetime(X['date_recorded'], infer_datetime_format=True)
  
  return X

In [140]:
X = little_wrangle(X)

In [124]:
X.head()

,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999-01-01,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010-01-01,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009-01-01,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986-01-01,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,NaT,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


We already have a test set for this model, the `test.csv` that you'll use to make the predictions you'll upload to Kaggle. However, since our competition only allows for 2 submissions per day, we need a way to estimate our training error so that we can quickly iterate and improve our model. We can achieve this by creating a validation set from the data we have.

**Task 4:** Split `X` and `y` into training and validation sets. Your validation set should be 20% of the data you have. You should have four variables: `X_train`, `X_val`, `y_train`, and `y_val`. 

In [141]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    train_size = 0.80,
    test_size = 0.20,
    stratify = train_val[target],
    random_state = 42,
    )
print('Shapes:', X_train.shape, X_val.shape, y_train.shape, y_val.shape)

Shapes: (47520, 39) (11880, 39) (47520,) (11880,)


**Test 4**

In [142]:
'''Task 4 Testing'''

assert X_train.shape == (47520, 39), '`X_train` is the wrong shape.'
assert X_val.shape == (11880, 39), '`X_val` is the wrong shape.'
assert y_train.shape == (47520,), '`y_train` is the wrong shape.'
assert y_val.shape == (11880,), '`y_val` is the wrong shape.'

## Establish Baseline

**Task 5:** This is a **classification** problem, so you need to establish the baseline accuracy for your training set. Find the majority class for `y_train` and calculate the percentage of labels in `y_train` belonging to that class. Assign your answer to the variable name `baseline_acc`.

In [143]:
# This method also works
# mode = y_train.mode()[0]
# baseline_acc = (y_train == mode).sum()/len(y_train)

baseline_acc = train['status_group'].value_counts(normalize=True).max()

print('Baseline Accuracy:', baseline_acc)

Baseline Accuracy: 0.5430765993265994


**Task 5**

In [144]:
'''Task 5 Testing'''

assert isinstance(baseline_acc, float), '`baseline_acc` should be a `float`.'
assert 0.0 <= baseline_acc <= 1.0, '`baseline_acc` is a score that should be between 0 and 1.'

## Build Model

**Task 6:** Create a model named `model` and train it with your training data. Your model should be a pipeline with (a) transformers that you think are appropriate to this dataset and (b) a `DecisionTreeClassifier` as your predictor. **Tip:** How can you transform categorical features and missing values in order to train your model?

In [145]:
from category_encoders import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier

# Encode the data and fit a model
model = Pipeline(steps=[
    ('ordinalencoder', OrdinalEncoder(return_df=True, handle_unknown='return_nan', handle_missing='return_nan')),
    ('imputer', SimpleImputer()),
    ('scaler', StandardScaler()),
    ('decisiontreeclassifier', DecisionTreeClassifier(
        max_depth=15,
        min_samples_leaf=15,
        # random_state=42,
        ),
    ),
    ])
model.fit(X_train_encoded, y_train)

Pipeline(memory=None,
         steps=[('ordinalencoder',
                 OrdinalEncoder(cols=[], drop_invariant=False,
                                handle_missing='return_nan',
                                handle_unknown='return_nan', mapping=[],
                                return_df=True, verbose=0)),
                ('imputer',
                 SimpleImputer(add_indicator=False, copy=True, fill_value=None,
                               missing_values=nan, strategy='mean',
                               verbose=0)),
                ('scaler',
                 StandardScaler(copy=True, with_mean=Tru...td=True)),
                ('decisiontreeclassifier',
                 DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                        criterion='gini', max_depth=15,
                                        max_features=None, max_leaf_nodes=None,
                                        min_impurity_decrease=0.0,
                                

**Test 6**

In [146]:
'''Task 6 Testing'''

assert len(model) > 1, 'Your model pipeline should have multiple steps.'
assert isinstance(model[-1], DecisionTreeClassifier), 'Your pipline should end in a `DecisionTreeClassifier`.'
assert hasattr(model, 'classes_'), 'Have you fit your model?'

## Check Metrics

**Task 7:** Calculate the training and validation accuracy of your model, and assign them to the valiables `training_acc` and `validation_acc`, respectively. Your validation accuracy should be greater than your baseline accuracy.

In [147]:
# Evaluate the model
training_acc = model.score(X_train_encoded, y_train)
validation_acc = model.score(X_val_encoded, y_val)
print('Training Accuracy: %.3f' % training_acc)
print('Validation Accuracy: %.3f' % validation_acc)

# Combine these two into a "Cautious Accuracy"
# Take the difference between the train and val scores and subtract that from val_score
# Rewards a high val score, but only if the model is not overfit.
cautious_score = 2 * val_score - training_acc
print('Cautious Accuracy: %.3f' % cautious_score)

Training Accuracy: 0.809
Validation Accuracy: 0.771
Cautious Accuracy: 0.733


**Test 7**

In [148]:
'''Task 7 Testing'''

assert isinstance(training_acc, float)
assert isinstance(validation_acc, float)
assert 0.0 <= training_acc <= 1.0
assert 0.0 <= validation_acc <= 1.0

## Create Kaggle Submission

**Task 8:** Load `'waterpumps/test.csv'` into a DataFrame named `X_test`. Generate a list of predictions, and then put them into a DataFrame `submission`. Be sure that `submission` has the same index as `X_test` and that the column name for your predictions is `'status_group'`.  

In [150]:
X_test = pd.read_csv(DATA_PATH + 'waterpumps/test_features.csv', index_col='id')
X_test = little_wrangle(X_test)

y_pred = model.predict(X_test)
submission = pd.DataFrame({'id': test.id, 'status_group': y_pred})

submission.head()

TypeError: ignored

**Test 8**

In [ ]:
'''Task 8 Testing'''

assert isinstance(submission, pd.DataFrame), '`submission` should be a DataFrame.'
assert len(submission) == 14358, '`submission` should have 14358 rows.'

**Task 9 (`stretch goal`):** Save `submission` as a csv file using [`.to_csv()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_csv.html) and submit it to our Kaggle competition.

In [ ]:
# Create the test prediction and submission file
make_submission(pipeline.predict(X_test),'JS_tree_15_15')

In [ ]:
from google.colab import files
def make_submission(y_pred, message):
  submission = pd.DataFrame({'id': test.id, 'status_group': y_pred})
  submission_filename = 'submission.csv'
  submission.to_csv(submission_filename, index=False)
  files.download(submission_filename)
  # !kaggle competitions submit -c dspt-pump-it-up-challenge -f submission.csv -m "{message}"

## Explain

**Task 10 (`stretch goal`):** Make a horizontal barchart of the 10 most important features for your model.

In [ ]:
import matplotlib.pyplot as plt

# YOUR CODE HERE
raise NotImplementedError()

In [159]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import datetime

# Retrieve data
train_val = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                     pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')

target = 'status_group'

# Wrangle train, validate, and test sets in the same way
def wrangle(X):
    
    # Prevent SettingWithCopyWarning
    X = X.copy()

    if target in X.columns:
      X = X.drop(columns=[target])
    
    # About 3% of the time, latitude has small values near zero,
    # outside Tanzania, so we'll treat these values like zero.
    X['latitude'] = X['latitude'].replace(-2e-08, 0)
    
    # When columns have zeros and shouldn't, they are like null values.
    # So we will replace the zeros with nulls, and impute missing values later.
    # Also create a "missing indicator" column, because the fact that
    # values are missing may be a predictive signal.
    cols_with_zeros = ['longitude', 'latitude', 'construction_year', 
                       'gps_height', 'population']
    for col in cols_with_zeros:
        X[col] = X[col].replace(0, np.nan)
        X[col+'_MISSING'] = X[col].isnull()
            
    # Drop duplicate columns
    duplicates = ['quantity_group', 'payment_type']
    X = X.drop(columns=duplicates)
    
    # Drop recorded_by (never varies) and id (always varies, random)
    unusable_variance = ['recorded_by', 'id']
    X = X.drop(columns=unusable_variance)
    
    # Convert date_recorded to datetime
    X['date_recorded'] = pd.to_datetime(X['date_recorded'], infer_datetime_format=True)
    
    # Extract components from date_recorded, then drop the original column
    X['year_recorded'] = X['date_recorded'].dt.year
    X['month_recorded'] = X['date_recorded'].dt.month
    X['day_recorded'] = X['date_recorded'].dt.day
    X = X.drop(columns='date_recorded')
    
    # Engineer feature: how many years from construction_year to date_recorded
    X['years'] = X['year_recorded'] - X['construction_year']
    X['years_MISSING'] = X['years'].isnull()
  
    return X

# Split train & val
# train, val = train_test_split(
#     train_val,
#     train_size = 0.80,
#     test_size = 0.20,
#     stratify = train_val[target],
#     random_state = train_val_seed,
#     )

# Actually wrangle and split the data up
X_train_val, y_train_val = wrangle(train_val), train_val[target]
X_test                   = wrangle(test)  # y_test is not provided

# Actually wrangle and split the data up
# X_train, y_train = wrangle(train), train[target]
# X_val, y_val     = wrangle(val), val[target]
# X_test           = wrangle(test)  # y_test is not provided

# print('Shapes:')
# print('Train, X_train, y_train:', train.shape, X_train.shape, y_train.shape)
# print('Val, X_val, y_val:', val.shape, X_val.shape, y_val.shape)
# print('Test, X_test:', test.shape, X_test.shape)

from category_encoders import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

# Make a pipeline for encoding
encoder = Pipeline(steps=[
    ('ordinalencoder', OrdinalEncoder(return_df=True, handle_unknown='return_nan', handle_missing='return_nan')),
    ('imputer', SimpleImputer()),
    ('scaler', StandardScaler()),
    ])
X_train_val_encoded = encoder.fit_transform(X_train_val)
# X_val_encoded = encoder.transform(X_val)
X_test_encoded = encoder.transform(X_test)

In [174]:
def try_model(train_val_seed=42, max_depth=15, n_estimators=10, min_samples=2):

    # Split train & val
    X_train_encoded, X_val_encoded = train_test_split(
        X_train_val_encoded,
        train_size = 0.80,
        test_size = 0.20,
        stratify = train_val[target],
        random_state = train_val_seed,
        )

    # Make and fit the model
    model = Pipeline(steps=[
        ('randomforestclassifier', RandomForestClassifier(
            max_depth = max_depth,
            n_estimators = n_estimators,
            min_samples_split = min_samples,
            ), 
        ),
        ])
    model.fit(X_train_encoded, y_train)

    # Evaluate the model
    train_score = model.score(X_train_encoded, y_train)
    val_score = model.score(X_val_encoded, y_val)
    # print('Train Accuracy: %.3f' % train_score)
    # print('Validation Accuracy: %.3f' % val_score)

    # Combine these two into a "Cautious Accuracy"
    # Take the difference between the train and val scores and subtract that from val_score
    # Rewards a high val score, but only if the model is not overfit.
    cautious_score = 2 * val_score - train_score
    # print('Cautious Accuracy: %.3f' % cautious_score)
    return cautious_score

In [176]:
for min_samples in range(2,33,10):
  for depth in range(8,14,4):
    scores = [try_model(train_val_seed=seed, max_depth=depth, n_estimators=10, min_samples=min_samples) for seed in range(5)]
    mean_score = np.array(scores).mean()
    print('Min Samples/Split =',min_samples,'Depth =',depth,'Mean Score =',mean_score)
    # print('Cautious Accuracy: %.3f' % cautious_score)

Min Samples/Split = 2 Depth = 8 Mean Score = 0.535976430976431
Min Samples/Split = 2 Depth = 12 Mean Score = 0.4923106060606061
Min Samples/Split = 12 Depth = 8 Mean Score = 0.536712962962963
Min Samples/Split = 12 Depth = 12 Mean Score = 0.5071717171717172
Min Samples/Split = 22 Depth = 8 Mean Score = 0.5380934343434343
Min Samples/Split = 22 Depth = 12 Mean Score = 0.5098779461279462
Min Samples/Split = 32 Depth = 8 Mean Score = 0.5382954545454545
Min Samples/Split = 32 Depth = 12 Mean Score = 0.518013468013468
